In [ ]:
import torch, os
from transformers import pipeline, BertForSequenceClassification, BertTokenizerFast
from torch.utils.data import Dataset

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
import shutil
import pandas as pd
import subprocess

# Extract 7z file using 7z command-line tool
def extract_7z(file_path):
    subprocess.run(['7z', 'x', file_path])

# Function to read text files from directories
def read_text_files(directory):
    text_data = []
    for subdir, dirs, files in os.walk(directory):
        if files:
            label = os.path.basename(subdir)
            for file in files:
                if file.endswith(".txt"):
                    file_path = os.path.join(subdir, file)
                    with open(file_path, 'r', encoding='utf-8') as f:
                        text = f.read()
                        text_data.append((text, label))
    return text_data

# Extract the 7z file
extract_7z("/content/cnn-arabic-utf8.7z")

# Read text files and create dataset
data = read_text_files("./cnn-arabic-utf8")

# Create DataFrame
df = pd.DataFrame(data, columns=['text', 'category'])

# Save DataFrame to CSV file
df.to_csv("dataset.csv", index=False)


In [ ]:
# Function to load data from folders
def load_data_from_folders(main_folder_path):
    data = []
    for category_folder in os.listdir(main_folder_path):
        category_label = category_folder  # Assuming folder name is the category label
        category_path = os.path.join(main_folder_path, category_folder)
        for file_name in os.listdir(category_path):
            file_path = os.path.join(category_path, file_name)
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()
                data.append({'text': content, 'category': category_label})
    return pd.DataFrame(data)

# Main folder containing subfolders for each category
main_folder_path = 'cnn-arabic-utf8'

# Load data from folders
df = load_data_from_folders(main_folder_path)

In [ ]:
labels = df['category'].unique().tolist()
labels = [s.strip() for s in labels ]
labels

['sport', 'middle_east', 'scitech', 'business', 'world', 'entertainment']

In [ ]:
StopWords = set(stopwords.words('arabic'))

ArabicDiacritics = r"""
                             ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                            ـ    | # Tatwil/Kashida
                     """

RegrexPattern = (
    "\U0001F600-\U0001F64F"+  # emoticons {😀 , 😆}
    "\U0001F300-\U0001F5FF"+  # symbols & pictographs {🌍 , 🌞}
    "\U0001F680-\U0001F6FF"+  # transport & map symbols {🚌 , 🚕 }
    "\U0001F1E0-\U0001F1FF"   # flags (iOS) { 🇺🇸 , 🇨🇦 }
)

In [ ]:
stop_words = list(StopWords)
stop_words[::50]

['أمّا',
 'ثم',
 'حمدا',
 'لم',
 'سحقا',
 'أكتوبر',
 'هل',
 'كلاهما',
 'كأيّن',
 'يفعلان',
 'إن',
 'ثاني',
 'تانِك',
 'يوليو',
 'ولو']

In [ ]:
def preprocessText(text):
    # Remove special characters {& $ @} and punctuation {. , ? !}
    text = re.sub(r'[^\w\s]', '', text)

    # Remove Arabic diacritics
    text = re.sub(ArabicDiacritics, '', text)

    # Remove emoji characters
    text = re.sub(f"[{RegrexPattern}]", '', text)

    text = re.sub('[A-Za-z]+', '', text)

    text = re.sub(r'[^\w\s\u0600-\u06FF]', '', text)
    text = re.sub(r'\d+', '', text)

    # Tokeniz The Sentence into tokens
    Tokens = word_tokenize(text)



    Tokens = [word for word in Tokens if word not in StopWords and len(word) > 1]

    PreprocessedText = ' '.join(Tokens)

    return PreprocessedText

In [ ]:
#test the preprocessing
Text1 = "مَرْحَبًا! كَيْفَ حَالُكَ الْيَوْمُ؟ 😀"
Text2 = "🤤🤤🤤 أنا أُحِبُّ الطَّعَامَ العَرَبِيَّ التَّقْلِيدِيَّ، مِثْلَ الْمُسَقَّعَةِ🤤🤤🤤🤤🤤🤤🤤 وَالفُلَّافِلِ."

preprocessedText1 = preprocessText(Text1)
preprocessedText2 = preprocessText(Text2)

print("Text 1 :", Text1)
print("Preprocessed Text 1:", preprocessedText1)
print("\nText2 :", Text2)
print("Preprocessed Text 2:", preprocessedText2)

Text 1 : مَرْحَبًا! كَيْفَ حَالُكَ الْيَوْمُ؟ 😀
Preprocessed Text 1: مرحبا حالك اليوم

Text2 : 🤤🤤🤤 أنا أُحِبُّ الطَّعَامَ العَرَبِيَّ التَّقْلِيدِيَّ، مِثْلَ الْمُسَقَّعَةِ🤤🤤🤤🤤🤤🤤🤤 وَالفُلَّافِلِ.
Preprocessed Text 2: أحب الطعام العربي التقليدي المسقعة والفلافل


In [ ]:
df['text'] = df['text'].apply(preprocessText)

In [ ]:
df.head()

,text,category
0,الترجي نهائي بطولة كأس أفريقيا الترجي نهائي بط...,sport
1,البرازيل لنهائي القارات بفوز صعب جنوب أفريقيا ...,sport
2,محكمة فرنسية تحكم لصالح اتحاد السيارات ضد فيرا...,sport
3,سيحضرون تقديم رونالدو البيرنابيو سيحضرون تقديم...,sport
4,الإنجليزي بيكام ميلانو بكأس الاتحاد الأوروبي ا...,sport


In [ ]:
CleanData = df[['text', 'category']]

In [ ]:
CleanData.sample(5)

,text,category
3421,روسيا وأوكرانيا تتفقان استئناف إمدادات الغاز ل...,business
1726,ردا إخلائهم بالقوة مستوطنون يثيرون الشغب بالخل...,middle_east
216,مدريد وافق التعاقد هونتيلار مقابل مليون مدريد ...,sport
3110,إرجاء افتتاح برج دبي المقبل الخميس الثانينوفمب...,business
2009,اليمن مقتل حوثيا سلفيين باشتباكات بصعدة الأربع...,middle_east


In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
CleanData['label'] = label_encoder.fit_transform(CleanData['category'])

LabelMapping = dict(zip(CleanData['label'], CleanData['category']))
print(f"Labels With Category :\n{LabelMapping}")

Labels With Category :
{4: 'sport', 2: 'middle_east', 3: 'scitech', 0: 'business', 5: 'world', 1: 'entertainment'}


In [ ]:
CleanData.sample(10)

,text,category,label
2832,أسهم الكويت توقف خسائرها والإمارات ومصر تتكبدا...,business,0
2505,الجينات وراء إصابة الأولاد بالتوحد البنات الجي...,scitech,3
3698,محقق منفذو هجمات مومباي قدموا باكستان محقق منف...,world,5
797,الحضري رسميا الدراويش لثلاث سنوات الحضري رسميا...,middle_east,2
4758,الروائي الفرنسي لوكليزيو يفوز بجائزة نوبل للآد...,entertainment,1
3571,الذهب يتراجع للأونصة الذهب يتراجع للأونصة احتم...,business,0
1352,قنبلة تصيب وزير الداخلية الصومالي وتقتل مساعدي...,middle_east,2
1394,نصر الله حرب إسرائيلية جديدة ستبدأ حيفا السبت ...,middle_east,2
2078,لجنة الانتخابات الفلسطينية توصي بتأجيلها لاستح...,middle_east,2
1679,قتلى جريحا حصيلة انفجارات متفرقة بالعراق الأحد...,middle_east,2


In [ ]:
df= CleanData[['category', 'text']]
df.sample(10)

,category,text
1728,middle_east,إشارات متضاربة بشأن نتائج الحوار الفلسطيني الق...
4649,entertainment,الختان وميتال مصر ويهود لبنان وحلاقة للقذافي ا...
581,sport,الإسماعيلي يدك مرمى الزمالك بالثلاثة ويتصدر ال...
4608,entertainment,مدونات تهمة التفكير بالسعودية وسرطان التحرش بم...
1614,middle_east,أسبوعان جديدان نتنياهو لتشكيل الحكومة الإسرائي...
1909,middle_east,بغداد تصر تدويل أزمتها سوريا وتحظى بدعم الجوار...
4873,entertainment,اختبار أثبت الرفات اختفى هتلر الخميس الثانينوف...
2305,scitech,سيوز الروسية تنطلق وعلى متنها رواد لمحطة الفضا...
1560,middle_east,صحف اتهامات فتحاوية لفياض بالتحضير لانقلاب أبي...
736,sport,الأرجنتين تتلقى الخسارة الأقسى عاما الأرجنتين ...


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from torch.utils.data import TensorDataset, DataLoader
from torch import optim
import torch.nn.functional as F
import torch
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"{torch.cuda.get_device_name(0)} GPU is used")
    !nvidia-smi
else:
    print("No GPU is detected")
    device = torch.device("cpu")

Tesla T4 GPU is used
Fri May  3 00:47:43 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8              11W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----

In [ ]:
model_name = "aubmindlab/bert-base-arabertv02"
tokenizer = AutoTokenizer.from_pretrained(model_name)
bert_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=11)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/381 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/825k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.64M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv02 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

target_encoder = LabelEncoder().fit(df.category)
target = target_encoder.transform(df.category)

X_train, X_test, y_train, y_test = train_test_split(df.drop(["category"], axis=1), target, test_size=0.2, random_state=10, stratify=target, shuffle=True)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=10, stratify=y_train, shuffle=True)

In [ ]:
MAX_LEN = 256

train_encoding = tokenizer(X_train["text"].to_list(), padding=True, truncation=True, max_length=MAX_LEN)
val_encoding = tokenizer(X_val["text"].to_list(), padding=True, truncation=True, max_length=MAX_LEN)
test_encoding = tokenizer(X_test["text"].to_list(), padding=True, truncation=True, max_length=MAX_LEN)

tr_input_ids, tr_attn_mask = torch.tensor(train_encoding.input_ids), torch.tensor(train_encoding.attention_mask)
val_input_ids, val_attn_mask = torch.tensor(val_encoding.input_ids), torch.tensor(val_encoding.attention_mask)
ts_input_ids, ts_attn_mask = torch.tensor(test_encoding.input_ids), torch.tensor(test_encoding.attention_mask)

In [ ]:
BATCH_SIZE = 32

train_data = TensorDataset(tr_input_ids, tr_attn_mask, torch.tensor(y_train))
train_dataloader = DataLoader(train_data, batch_size=BATCH_SIZE)

val_data = TensorDataset(val_input_ids, val_attn_mask, torch.tensor(y_val))
val_dataloader = DataLoader(val_data, batch_size=BATCH_SIZE)

test_data = TensorDataset(ts_input_ids, ts_attn_mask, torch.tensor(y_test))
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE)

In [ ]:
optimizer = optim.AdamW(bert_model.parameters(), lr=1e-5, weight_decay=0.02)
lr_scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.4)

In [ ]:
import numpy as np

def flat_accuracy(y_pred, y):
    flat_y_pred = np.argmax(y_pred, axis=1)
    return np.sum(flat_y_pred == y) / len(y)

EPOCHS = 3
bert_model.to(device)
for e in range(EPOCHS):
    print("="*40)
    print(f'Epoch: {e+1}/{EPOCHS} | lr: {optimizer.param_groups[0]["lr"]}')
    print("="*40)
    bert_model.train()

    train_accuracy = []
    # going through batches
    for s, batch in enumerate(train_dataloader):
        input_ids, attn_mask, y = batch
        optimizer.zero_grad()

        # forward
        output = bert_model(input_ids.to(device), attention_mask=attn_mask.to(device), labels=y.to(device))

        # backward
        output.loss.backward()

        # update params
        optimizer.step()

        # tracking vars
        accuracy = flat_accuracy(output.logits.detach().cpu().numpy(), y.to("cpu").numpy())
        train_accuracy.append(accuracy)

        print(f'Step: {s:^5} | Loss: {output.loss.item()}')

    print("-"*40)
    print(f'Train Accuracy: {sum(train_accuracy) / len(train_accuracy) * 100:.2f} %')
    lr_scheduler.step()

    bert_model.eval()
    val_accuracy = []
    for batch in val_dataloader:
        input_ids, attn_mask, y = batch
        with torch.no_grad():
            output = bert_model(input_ids.to(device), attention_mask=attn_mask.to(device))

        accuracy = flat_accuracy(output.logits.detach().cpu().numpy(), y.to("cpu").numpy())
        val_accuracy.append(accuracy)
    print(f"Val Accuracy: {sum(val_accuracy) / len(val_accuracy) * 100:.2f} %")
    print("-"*40)

bert_model.save_pretrained("path/to/sequence-classification-model")

Epoch: 1/3 | lr: 1e-05
Step:   0   | Loss: 2.5063846111297607
Step:   1   | Loss: 2.667219877243042
Step:   2   | Loss: 2.472419500350952
Step:   3   | Loss: 2.366488218307495
Step:   4   | Loss: 2.2464966773986816
Step:   5   | Loss: 2.2649271488189697
Step:   6   | Loss: 2.135798215866089
Step:   7   | Loss: 2.333360433578491
Step:   8   | Loss: 2.290102005004883
Step:   9   | Loss: 2.2763900756835938
Step:  10   | Loss: 2.1700048446655273
Step:  11   | Loss: 1.9110745191574097
Step:  12   | Loss: 1.9461442232131958
Step:  13   | Loss: 1.8493419885635376
Step:  14   | Loss: 1.9143989086151123
Step:  15   | Loss: 1.7274903059005737
Step:  16   | Loss: 1.773383617401123
Step:  17   | Loss: 1.959263563156128
Step:  18   | Loss: 1.7671486139297485
Step:  19   | Loss: 1.6579545736312866
Step:  20   | Loss: 1.8641080856323242
Step:  21   | Loss: 1.6177756786346436
Step:  22   | Loss: 1.6578210592269897
Step:  23   | Loss: 1.612115502357483
Step:  24   | Loss: 1.5229164361953735
Step:  25  

In [ ]:
def bert_model_predict(model, dataloader, score="flat"):
    bert_model.eval()
    pred_probs = []
    logits_list = []

    for batch in dataloader:
        input_ids, attn_mask, _ = batch

        with torch.no_grad():
            logits = model(input_ids.to(device), attn_mask.to(device)).logits

        probs = F.softmax(logits, dim=1)
        pred_probs.append(probs)
        logits_list.append(logits)

    pred_probs = torch.cat(pred_probs).cpu().numpy()

    if score=="probs":
        return pred_probs
    elif score=="logits":
        return torch.cat(logits_list).cpu().numpy()
    else:
        pred_flat = np.argmax(pred_probs, axis=1)
        return pred_flat

In [ ]:
bert_pred_flat = bert_model_predict(bert_model, test_dataloader, score="flat")
print(classification_report(y_test ,bert_pred_flat, labels=target, target_names=target_encoder.inverse_transform([0,1,2,3,4,5])))

               precision    recall  f1-score   support

     business       0.99      1.00      0.99       152
entertainment       0.99      1.00      0.99       152
  middle_east       0.99      1.00      0.99       152
      scitech       0.99      1.00      0.99       152
        sport       0.99      1.00      0.99       152
        world       0.99      1.00      0.99       152

     accuracy                           0.94    988082
    macro avg       0.94      0.94      0.94    988082
 weighted avg       0.94      0.95      0.94    988082



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:2326: UserWarning: labels size, 5070, does not match size of target_names, 6
  warnings.warn(


In [ ]:
def evaluate_single_instance(model, tokenizer, text):
    encoding = tokenizer(text, padding=True, truncation=True, max_length=MAX_LEN, return_tensors='pt')
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    with torch.no_grad():
        logits = model(input_ids, attention_mask=attention_mask).logits
    probs = torch.softmax(logits, dim=1).cpu().numpy()[0]
    return probs

# Interactive evaluation loop
while True:
    text = input("Enter text to evaluate (type 'quit' to exit): ")
    if text.lower() == 'quit':
        break
    probabilities = evaluate_single_instance(bert_model, tokenizer, text)
    predicted_class = np.argmax(probabilities)
    predicted_label = target_encoder.inverse_transform([predicted_class])[0]
    print(f"Predicted class: {predicted_label}")
    for label, prob in zip(target_encoder.classes_, probabilities):
        print(f"{label}: {prob:.2f}")


Enter text to evaluate (type 'quit' to exit): "أدت عدم اليقين الاقتصادي إلى انخفاض في سوق الأسهم، مما دفع الشركات إلى إعادة التقييم لاستراتيجياتها في العام القادم."
Predicted class: sport
business: 0.15
entertainment: 0.11
middle_east: 0.07
scitech: 0.11
sport: 0.21
world: 0.01
Enter text to evaluate (type 'quit' to exit): "ألحقت إصابة النجم ظلامًا بفرص الفريق في البطولة القادمة."
Predicted class: sport
business: 0.04
entertainment: 0.14
middle_east: 0.05
scitech: 0.13
sport: 0.40
world: 0.01
Enter text to evaluate (type 'quit' to exit): "تعثرت المفاوضات الدبلوماسية بين الدول المجاورة، مما أدى إلى تعطيل الجهود في حل النزاعات الإقليمية المستمرة."
Predicted class: sport
business: 0.02
entertainment: 0.09
middle_east: 0.09
scitech: 0.07
sport: 0.23
world: 0.02
Enter text to evaluate (type 'quit' to exit): quit
